# Imports


In [27]:
import random
import gradio as gr


# Flower



In [40]:
class Flower:
    def __init__(self, sizeCenter , colorCenter , colorPetals , colorStem , numPetals): 
        self.sizeCenter = sizeCenter
        self.colorCenter = colorCenter
        self.colorPetals = colorPetals
        self.colorStem = colorStem
        self.numPetals = numPetals
        
    def display(self):
        print(f"------------- Flower Details: -------------")
        print(f"Center Size: {self.sizeCenter}")
        print(f"Center Color: {self.colorCenter}")
        print(f"Petals Color: {self.colorPetals}")
        print(f"Stem Color: {self.colorStem}")
        print(f"Number of Petals: {self.numPetals}")



# Population


In [42]:
# Assume sizeCenter is 8 bits 
# Assume numPetals is 8 bits
# we may change the colorCenter, colorPetals, colorStem to be 24 bits each (8 bits for R, 8 bits for G, 8 bits for B)

def createPopulation():
    population = []  
    for i in range(8):
        sizeCenter = random.randint(0, 255)
        colorCenter = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        colorPetals = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        colorStem = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        numPetals = random.randint(0, 7)
        flower = Flower(sizeCenter, colorCenter, colorPetals, colorStem, numPetals)
        population.append(flower)
    return population

population = createPopulation()
population[0].display()
population[1].display()
population[2].display()
population[3].display()
population[4].display()
population[5].display()
population[6].display()
population[7].display()

------------- Flower Details: -------------
Center Size: 155
Center Color: (38, 167, 54)
Petals Color: (48, 16, 45)
Stem Color: (21, 51, 98)
Number of Petals: 7
------------- Flower Details: -------------
Center Size: 109
Center Color: (66, 35, 224)
Petals Color: (255, 241, 119)
Stem Color: (172, 199, 177)
Number of Petals: 7
------------- Flower Details: -------------
Center Size: 174
Center Color: (159, 115, 118)
Petals Color: (14, 45, 22)
Stem Color: (189, 96, 183)
Number of Petals: 3
------------- Flower Details: -------------
Center Size: 58
Center Color: (125, 237, 243)
Petals Color: (155, 50, 221)
Stem Color: (138, 210, 81)
Number of Petals: 6
------------- Flower Details: -------------
Center Size: 208
Center Color: (228, 38, 207)
Petals Color: (162, 146, 204)
Stem Color: (15, 78, 45)
Number of Petals: 1
------------- Flower Details: -------------
Center Size: 48
Center Color: (59, 145, 135)
Petals Color: (205, 209, 55)
Stem Color: (140, 25, 55)
Number of Petals: 5
------------

In [43]:
def convertDecimalToBinary(n):
    return bin(n).replace("0b", "").zfill(8)

def convertDecimalToBinary3(n):
    return bin(n).replace("0b", "").zfill(3)

print(convertDecimalToBinary(7))


00000111


In [44]:
def displayBinaryRepresentation(flower):
    sizeCenterBinary = convertDecimalToBinary(flower.sizeCenter)
    colorCenterBinary = ''.join([convertDecimalToBinary(c) for c in flower.colorCenter])
    colorPetalsBinary = ''.join([convertDecimalToBinary(c) for c in flower.colorPetals])
    colorStemBinary = ''.join([convertDecimalToBinary(c) for c in flower.colorStem])
    numPetalsBinary = convertDecimalToBinary3(flower.numPetals)
    
    chromosome = sizeCenterBinary + colorCenterBinary + colorPetalsBinary + colorStemBinary + numPetalsBinary
    print(f"Chromosome: {chromosome}")

def displayAllChromosomes(population):
    for i, flower in enumerate(population):
        print(f"Flower {i+1}:")
        displayBinaryRepresentation(flower)
        print()

displayAllChromosomes(population)

Flower 1:
Chromosome: 10011011001001101010011100110110001100000001000000101101000101010011001101100010111

Flower 2:
Chromosome: 01101101010000100010001111100000111111111111000101110111101011001100011110110001111

Flower 3:
Chromosome: 10101110100111110111001101110110000011100010110100010110101111010110000010110111011

Flower 4:
Chromosome: 00111010011111011110110111110011100110110011001011011101100010101101001001010001110

Flower 5:
Chromosome: 11010000111001000010011011001111101000101001001011001100000011110100111000101101001

Flower 6:
Chromosome: 00110000001110111001000110000111110011011101000100110111100011000001100100110111101

Flower 7:
Chromosome: 10010111111001010100111000111100010100000111000010110110110101101111000011011010101

Flower 8:
Chromosome: 11001101101100001111111000010000001111101110001001011100000101010000111000001110100



In [45]:
def convertBinaryToDecimal(chromosome):
    decimal = 0
    for i in range(len(chromosome)):
        decimal += int(chromosome[i]) * (2 ** (len(chromosome) - 1 - i))
    return decimal
   

In [46]:
print(convertBinaryToDecimal("00110001"))

49


In [47]:
def repair(min_value, max_value,x_value):
    return max(min_value, min(max_value, x_value))

# min 0
# max 1000

# num > 1000 2000
# num < 0  -1000

In [ ]:
# Create population  
# Create Flower 
# Selection 
# Crossover
# Mutation
# Fitness Function
# Main 
# GUI 

# Selection
this is not crossover


In [48]:
def get_parents(flower):
    sizeCenterBinary = convertDecimalToBinary(flower.sizeCenter)
    colorCenterBinary = ''.join([convertDecimalToBinary(c) for c in flower.colorCenter])
    colorPetalsBinary = ''.join([convertDecimalToBinary(c) for c in flower.colorPetals])
    colorStemBinary = ''.join([convertDecimalToBinary(c) for c in flower.colorStem])
    numPetalsBinary = convertDecimalToBinary(flower.numPetals)
    
    chromosome = sizeCenterBinary + colorCenterBinary + colorPetalsBinary + colorStemBinary + numPetalsBinary
    return chromosome

def selection(population):
    fitness_values = []
    for flower in population:
        chromosome = get_parents(flower)
        #fitness is based on time spent by mouse hovering on flower
        fitness_value = convertBinaryToDecimal(chromosome)
        fitness_values.append((fitness_value, flower))
    
    # Sort by fitness value in descending order
    fitness_values.sort(reverse=True, key=lambda x: x[0])
    
    ## this is elitism, we want to make roulette which is that each the higher the fitness fn, the higher its probability is to be chosen.
    ## i think we need to normalize the probabilities then get a random value from 0 to 1 generated and if it returns 0.4 we see which area this returns to.
    # Select top 65% as parents
    num_parents = int(len(population) * 0.65)
    parents = [flower for _, flower in fitness_values[:num_parents]]
    
    return parents

parents = selection(population)
for i, parent in enumerate(parents):
    print(f"Parent {i+1}:")
    parent.display()
    print("\n")


Parent 1:
------------- Flower Details: -------------
Center Size: 208
Center Color: (228, 38, 207)
Petals Color: (162, 146, 204)
Stem Color: (15, 78, 45)
Number of Petals: 1


Parent 2:
------------- Flower Details: -------------
Center Size: 205
Center Color: (176, 254, 16)
Petals Color: (62, 226, 92)
Stem Color: (21, 14, 14)
Number of Petals: 4


Parent 3:
------------- Flower Details: -------------
Center Size: 174
Center Color: (159, 115, 118)
Petals Color: (14, 45, 22)
Stem Color: (189, 96, 183)
Number of Petals: 3


Parent 4:
------------- Flower Details: -------------
Center Size: 155
Center Color: (38, 167, 54)
Petals Color: (48, 16, 45)
Stem Color: (21, 51, 98)
Number of Petals: 7


Parent 5:
------------- Flower Details: -------------
Center Size: 151
Center Color: (229, 78, 60)
Petals Color: (80, 112, 182)
Stem Color: (214, 240, 218)
Number of Petals: 5




# Crossover


In [49]:
def crossover( flower1, flower2):
    crossPos = random.randint(1, len(flower1)-1)
    # print(crossPos)

    # Perform crossover
    offspring1 = flower1[:crossPos] + flower2[crossPos:]
    offspring2 = flower2[:crossPos] + flower1[crossPos:]
    return offspring1, offspring2

flower3 = "011100000101"
flower4 = "100101101100"
print(crossover(flower3,flower4))


('011101101100', '100100000101')


# Mutation

In [ ]:
def mutation(population):
    chromosome = get_parents(population[0])
    total_bits = len(chromosome) * len(population)
    print(total_bits)
    num_mutations = int(total_bits * 0.05)  # 5% of the total bits
    i = 0
    print(f"Total bits: {total_bits}, Number of mutations to perform: {num_mutations}\n")  

    for i in range(num_mutations):
        flower_index = random.randint(0, len(population) - 1)
        print(f"Mutation {i + 1}: Flower {flower_index + 1}")
        flower = population[flower_index]
        chromosome = get_parents(flower)
        
        mutation_pos = random.randint(0, len(chromosome) - 1)
        if chromosome[mutation_pos] == '0':
            mutated_bit = '1'
        else:
            mutated_bit = '0'
        new_chromosome = chromosome[:mutation_pos] + mutated_bit + chromosome[mutation_pos + 1:]

        print(f"Flower {flower_index + 1} mutated at position {mutation_pos}:")
        print(f"Old Chromosome: {chromosome}")
        print(f"New Chromosome: {new_chromosome}\n")

        # ------------------- My code ends here this is copilot suggestion -------------------

        # Update flower attributes based on new chromosome
        flower.sizeCenter = convertBinaryToDecimal(new_chromosome[0:8])
        flower.colorCenter = (
            convertBinaryToDecimal(new_chromosome[8:16]),
            convertBinaryToDecimal(new_chromosome[16:24]),
            convertBinaryToDecimal(new_chromosome[24:32])
        )
        flower.colorPetals = (
            convertBinaryToDecimal(new_chromosome[32:40]),
            convertBinaryToDecimal(new_chromosome[40:48]),
            convertBinaryToDecimal(new_chromosome[48:56])
        )
        flower.colorStem = (
            convertBinaryToDecimal(new_chromosome[56:64]),
            convertBinaryToDecimal(new_chromosome[64:72]),
            convertBinaryToDecimal(new_chromosome[72:80])
        )
        flower.numPetals = repair(0, 7, convertBinaryToDecimal(new_chromosome[80:83]))
mutation(population)

704
Total bits: 704, Number of mutations to perform: 35

Mutation 1: Flower 2
Flower 2 mutated at position 83:
Old Chromosome: 0110110101000010001000111110000011111111111100010111011110101100110001111011000100000111
New Chromosome: 0110110101000010001000111110000011111111111100010111011110101100110001111011000100010111

Mutation 2: Flower 2
Flower 2 mutated at position 85:
Old Chromosome: 0110110101000010001000111110000011111111111100010111011110101100110001111011000100000000
New Chromosome: 0110110101000010001000111110000011111111111100010111011110101100110001111011000100000100

Mutation 3: Flower 4
Flower 4 mutated at position 33:
Old Chromosome: 0011101001111101111011011111001110011011001100101101110110001010110100100101000100000110
New Chromosome: 0011101001111101111011011111001111011011001100101101110110001010110100100101000100000110

Mutation 4: Flower 1
Flower 1 mutated at position 70:
Old Chromosome: 100110110010011010100111001101100011000000010000001011010001010100110011011000

# GUI

In [28]:
def greet(name):
    return f"Hello {name}!"

app = gr.Interface(fn=greet, inputs="text", outputs="text")
app.launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Created dataset file at: .gradio\flagged\dataset1.csv
